In [2]:
# import pandas as pd

# # Load your data
# df = pd.read_csv("sensor_data_formatted_fire.csv")

# # Confirm column names
# print("✅ Columns:", df.columns)

# def classify(row):
#     t = row["Temperature"]
#     g = row["Gas"]
#     f_status = row["Fire_Status"]
#     l = row["LDR"]

#     print(f"Checking row: Temperature={t}, Gas={g}, Fire_Status={f_status}, LDR={l}")  # Debugging line

#     # Condition breakdowns
#     temperature_condition = "Normal" if t < 45 else "High" if t <= 53 else "Very High"
#     gas_condition = "Normal" if g < 300 else "High" if g <= 399 else "Very High"
#     fire_condition = f"Status: {f_status}"
#     ldr_condition = "Normal" if l > 700 else "Low Brightness" if l > 300 else "Very Low Brightness"
    
#     print(f"Temperature condition: {temperature_condition}, Gas condition: {gas_condition}, Fire condition: {fire_condition}, LDR condition: {ldr_condition}")

#     # 🟡 MODERATE
#     moderate_cases = [
#         (200 <= g < 300 and f_status == "Weak Flame" and t < 45 and l == "Bright", "Gas level moderate with weak flame and bright light"),
#         (45 <= t <= 53 and 200 <= g < 300 and f_status == "Weak Flame" and l == "Bright", "Moderate gas and temperature with weak flame and bright light"),
#         (200 <= g < 300 and f_status == "Weak Flame" and l == "Bright", "Weak flame, moderate gas, and bright light"),
#     ]
#     for condition, reason in moderate_cases:
#         if condition:
#             print(f"Condition matched: 🟡 Moderate | Reason: {reason}")
#             return "🟡 Moderate", reason

#     # 🟠 HIGH
#     high_cases = [
#         (g >= 300 and f_status == "Weak Flame" and t >= 45 and l == "Bright", "High gas, weak flame, and elevated temperature with bright light"),
#         (g >= 300 and f_status == "Weak Flame" and l == "Bright", "High gas and weak flame with bright light"),
#         (g >= 300 and f_status == "Weak Flame" and t >= 45, "High gas, weak flame, and temperature above 45"),
#     ]
#     for condition, reason in high_cases:
#         if condition:
#             print(f"Condition matched: 🟠 High | Reason: {reason}")
#             return "🟠 High", reason

#     # 🔴 VERY HIGH
#     very_high_cases = [
#         (g >= 400 and f_status == "Strong Flame" and t >= 54 and l == "Very Low Brightness", "Extreme gas, strong flame, high temperature, and very low light"),
#         (t >= 54, "Extreme temperature, fire very high hazard"),
#     ]
#     for condition, reason in very_high_cases:
#         if condition:
#             print(f"Condition matched: 🔴 Very High | Reason: {reason}")
#             return "🔴 Very High", reason

#     # Normal conditions (Temperature 0-20)
#     if 0 <= t <= 20:
#         print(f"Condition matched: 🟢 Normal | Reason: Temperature is in the normal range (0-20°C).")
#         return "🟢 Normal", "Temperature is in the normal range (0-20°C)."

#     # Default case (catch-all)
#     print("No conditions matched, defaulting to 🟠 High")
#     return "🟠 High", "Unclassified anomaly (defaulted to High)"

# # Convert all sensor readings to numeric (if not already)
# df[["Temperature", "Gas", "Fire", "LDR"]] = df[["Temperature", "Gas", "Fire", "LDR"]].apply(pd.to_numeric, errors="coerce")

# # Apply classification to all rows
# df[["Status", "Reason"]] = df.apply(classify, axis=1, result_type="expand")

# # Save to new CSV
# df.to_csv("sensor_datarec3.csv", index=False)
# print("✅ Labeled dataset saved as 'sensor_datarec2.csv'")


In [3]:
import pandas as pd

# Load your data
df = pd.read_csv("sensor_data_formatted_fire.csv")

# Confirm column names
print("✅ Columns:", df.columns)

# Convert Fire from hex to decimal and overwrite the same column
def hex_to_decimal(val):
    try:
        return int(str(val), 16)
    except:
        return 0  # fallback for bad or missing values

df["Fire"] = df["Fire"].apply(hex_to_decimal)

# Derive Fire_Status from decimal Fire value
def get_fire_status(fire_val):
    if fire_val < 300:
        return "Strong Flame"
    elif fire_val < 600:
        return "Weak Flame"
    else:
        return "No Flame"

df["Fire_Status"] = df["Fire"].apply(get_fire_status)

# Derive LDR brightness status
def get_ldr_status(lux):
    if lux > 700:
        return "Bright"
    elif lux > 300:
        return "Low Brightness"
    else:
        return "Very Low Brightness"

df["LDR_Status"] = df["LDR"].apply(get_ldr_status)

# Classification logic
def classify(row):
    t = row["Temperature"]
    g = row["Gas"]
    f_status = row["Fire_Status"]
    l_status = row["LDR_Status"]

    print(f"Checking row: Temp={t}, Gas={g}, Fire={f_status}, LDR={l_status}")

    # 🟡 MODERATE
    moderate_cases = [
        (200 <= g < 300 and f_status == "Weak Flame" and t < 45 and l_status == "Bright", "Gas level moderate with weak flame and bright light"),
        (45 <= t <= 53 and 200 <= g < 300 and f_status == "Weak Flame" and l_status == "Bright", "Moderate gas and temperature with weak flame and bright light"),
        (200 <= g < 300 and f_status == "Weak Flame" and l_status == "Bright", "Weak flame, moderate gas, and bright light"),
    ]
    for condition, reason in moderate_cases:
        if condition:
            return "🟡 Moderate", reason

    # 🟠 HIGH
    high_cases = [
        (g >= 300 and f_status == "Weak Flame" and t >= 45 and l_status == "Bright", "High gas, weak flame, and elevated temperature with bright light"),
        (g >= 300 and f_status == "Weak Flame" and l_status == "Bright", "High gas and weak flame with bright light"),
        (g >= 300 and f_status == "Weak Flame" and t >= 45, "High gas, weak flame, and temperature above 45"),
    ]
    for condition, reason in high_cases:
        if condition:
            return "🟠 High", reason

    # 🔴 VERY HIGH
    very_high_cases = [
        (g >= 400 and f_status == "Strong Flame" and t >= 54 and l_status == "Very Low Brightness", "Extreme gas, strong flame, high temperature, and very low light"),
        (t >= 54, "Extreme temperature, fire very high hazard"),
    ]
    for condition, reason in very_high_cases:
        if condition:
            return "🔴 Very High", reason

    # 🟢 Normal
    if 0 <= t <= 20:
        return "🟢 Normal", "Temperature is in the normal range (0-20°C)."

    # Catch-all fallback
    return "🟠 High", "Unclassified anomaly (defaulted to High)"

# Ensure all values are numeric
df[["Temperature", "Gas", "Fire", "LDR"]] = df[["Temperature", "Gas", "Fire", "LDR"]].apply(pd.to_numeric, errors="coerce")

# Apply classification logic
df[["Status", "Reason"]] = df.apply(classify, axis=1, result_type="expand")

# Save final dataset
df.to_csv("sensor_datarec3.csv", index=False)
print("✅ Labeled dataset saved as 'sensor_datarec3.csv'")


✅ Columns: Index(['Timestamp', 'Temperature', 'Gas', 'LDR', 'Fire', 'Temp_Status',
       'Gas_Status', 'LDR_Status', 'Fire_Status'],
      dtype='object')
Checking row: Temp=44, Gas=266, Fire=Weak Flame, LDR=Very Low Brightness
Checking row: Temp=29, Gas=272, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=50, Gas=248, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=42, Gas=202, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=47, Gas=255, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=50, Gas=213, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=37, Gas=226, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=48, Gas=298, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=3, Gas=274, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=16, Gas=280, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=22, Gas=291, Fire=No Flame, LDR=Very Low Brightness
Checking row: Temp=42, Gas=261, Fire=No Flame, LDR=Very Low Br

In [4]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import LabelEncoder
# from sklearn.impute import SimpleImputer
# import joblib

# # Load your labeled dataset
# df = pd.read_csv("sensor_datarec2.csv")

# # Confirm columns
# print("✅ Columns:", df.columns)

# # Impute missing values (mean for numerical features)
# imputer = SimpleImputer(strategy="mean")
# df[["Temperature", "Gas", "Fire", "LDR"]] = imputer.fit_transform(df[["Temperature", "Gas", "Fire", "LDR"]])

# # Encode the target labels (Status) using LabelEncoder
# label_encoder = LabelEncoder()
# df['Status_encoded'] = label_encoder.fit_transform(df['Status'])

# # Features (input variables)
# X = df[["Temperature", "Gas", "Fire", "LDR"]]  # Select your relevant columns

# # Target variable (encoded status)
# y = df['Status_encoded']  # Encoded target

# # Split into train and test sets (80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize and train the Random Forest classifier
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = clf.predict(X_test)

# # Print the confusion matrix and classification report
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# # Print feature importances
# feature_importances = clf.feature_importances_
# for feature, importance in zip(X.columns, feature_importances):
#     print(f"{feature}: {importance:.4f}")

# # Save the trained model to a file
# joblib.dump(clf, "random_forest_fire_model.pkl")

# # (Optional) Load the model for prediction
# # loaded_model = joblib.load("random_forest_fire_model.pkl")
# # new_data = [[30, 250, "Weak Flame", "Bright"]]  # Example new data
# # new_data_imputed = imputer.transform(new_data)  # Apply imputation to new data
# # new_prediction = loaded_model.predict(new_data_imputed)
# # decoded_prediction = label_encoder.inverse_transform(new_prediction)
# # print("Predicted Status:", decoded_prediction)


In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Step 1: Load your dataset
df = pd.read_csv("sensor_datarec3.csv")

# Drop non-numeric or non-feature columns
df = df.drop(columns=["Timestamp", "Temp_Status", "Gas_Status", "LDR_Status", "Fire_Status", "Reason"])

# Encode target labels (Status)
label_encoder = LabelEncoder()
df["Status_encoded"] = label_encoder.fit_transform(df["Status"])

# Step 2: Define features and target
X = df[["Temperature", "Gas", "LDR", "Fire"]]
y = df["Status_encoded"]

# Step 3: Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Optional: Save model for later use
import joblib
joblib.dump((model, label_encoder), "fire_safety_rf_model.pkl")


✅ Accuracy: 0.9950124688279302
✅ Classification Report:
               precision    recall  f1-score   support

 🔴 Very High       0.99      1.00      0.99       373
      🟠 High       1.00      0.99      0.99       260
  🟡 Moderate       1.00      0.50      0.67         2
    🟢 Normal       1.00      1.00      1.00       167

    accuracy                           1.00       802
   macro avg       1.00      0.87      0.91       802
weighted avg       1.00      1.00      0.99       802



['fire_safety_rf_model.pkl']

In [ ]:
import serial
import time
import joblib
import csv
from datetime import datetime

# Load model and label encoder
model, label_encoder = joblib.load("fire_safety_rf_model.pkl")

# Function to convert hex to decimal
def hexadecimalToDecimal(hexval):
    length = len(hexval)
    base = 1
    dec_val = 0
    for i in range(length - 1, -1, -1):
        if '0' <= hexval[i] <= '9':
            dec_val += (ord(hexval[i]) - 48) * base
        elif 'A' <= hexval[i] <= 'F':
            dec_val += (ord(hexval[i]) - 55) * base
        base = base * 16
    return dec_val

# Connect to Arduino
# ser = serial.Serial('COM5', 9600, timeout=1)
time.sleep(2)

print("🔁 Live Prediction Started... Press Ctrl+C to stop.")

try:
    while True:
        line = ser.read_until(b'E')
        line = line.decode('utf-8', errors='ignore').strip()

        if line.startswith('A') and 'B' in line and 'C' in line and 'D' in line:
            try:
                temp_str = line[line.index('A')+1:line.index('B')].strip()
                gas_str = line[line.index('B')+1:line.index('C')].strip()
                ldr_str = line[line.index('C')+1:line.index('D')].strip()
                fire_str_raw = line[line.index('D')+1:].strip()

                # Convert strings to integers
                temp = int(temp_str)
                gas = int(gas_str)
                ldr = int(ldr_str)
                fire = hexadecimalToDecimal(fire_str_raw)

                # Predict using loaded model
                input_data = [[temp, gas, ldr, fire]]
                prediction = model.predict(input_data)
                status = label_encoder.inverse_transform(prediction)[0]

                # Timestamp
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                # Print prediction
                print(f"{timestamp} | Temp: {temp} | Gas: {gas} | LDR: {ldr} | Fire: {fire} => 🔥 Predicted Status: {status}")

            except Exception as e:
                print("❌ Error parsing line:", line)
                print("Exception:", e)

except KeyboardInterrupt:
    print("\n🛑 Live prediction stopped by user.")
    ser.close()


🔁 Live Prediction Started... Press Ctrl+C to stop.
2025-04-18 13:13:27 | Temp: 31 | Gas: 151 | LDR: 506 | Fire: 2206 => 🔥 Predicted Status: 🟠 High
2025-04-18 13:13:27 | Temp: 39 | Gas: 151 | LDR: 506 | Fire: 862 => 🔥 Predicted Status: 🟠 High


c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


2025-04-18 13:13:28 | Temp: 44 | Gas: 151 | LDR: 506 | Fire: 622 => 🔥 Predicted Status: 🟠 High


c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


2025-04-18 13:13:28 | Temp: 45 | Gas: 150 | LDR: 506 | Fire: 654 => 🔥 Predicted Status: 🟠 High


...

2025-04-18 13:14:21 | Temp: 45 | Gas: 138 | LDR: 506 | Fire: 654 => 🔥 Predicted Status: 🟠 High


c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


2025-04-18 13:14:21 | Temp: 45 | Gas: 137 | LDR: 506 | Fire: 782 => 🔥 Predicted Status: 🟠 High


c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


2025-04-18 13:14:22 | Temp: 45 | Gas: 139 | LDR: 506 | Fire: 782 => 🔥 Predicted Status: 🟠 High

🛑 Live prediction stopped by user.


AttributeError: partially initialized module 'streamlit' has no attribute 'set_page_config' (most likely due to a circular import)